In [1]:
from torch.nn.utils.rnn import pad_sequence
from tfrecord_lite import tf_record_iterator
from tfrecord.torch.dataset import TFRecordDataset
import torch.utils.data as tdata
import torch

import json

In [2]:
block_config_path = '/scratch/gilbreth/akamsali/Research/Makin/ecog2txt-pytorch/conf/block_breakdowns.json'
tfrecord_path = '/depot/jgmakin/data/ecog2txt/word_sequence/tf_records'

subjects = ['400', '401', '402', '403']


with open(block_config_path) as bf:
    block_config_all = json.load(bf)

In [17]:
block_config = block_config_all['400']
filtered_files = list(
            map(lambda y: tfrecord_path + "/EFC400" + "_B" + y[0] + ".tfrecord", 
                 block_config.items()))

In [ ]:

files = {}
for subject_id in subjects:
    block_config = block_config_all[subject_id]

    keys = list(block_config.keys())
    files[subject_id] = list(map(lambda y: tfrecord_path + "/EFC" + subject_id + "_B" + y + ".tfrecord", keys))